# Install Dependencies

In [1]:
!pip install datasets peft -qq
!pip install accelerate -qq
!pip install bitsandbytes -qq

# Import Packages

In [3]:
from huggingface_hub import login
from datasets import load_dataset
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig)

from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [5]:
# Define the model ID for the pre-trained model
model_id = "WiroAI/WiroAI-Finance-Qwen-1.5B"

# Configure BitsAndBytes for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model with the specified quantization configuration and device mapping
base_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

# Check if the tokenizer has a pad token defined
if tokenizer.pad_token is None:
  # If not, assign the end of sequence (eos) token as the pad token
  tokenizer.pad_token = tokenizer.eos_token

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [7]:
import json

# Sample JSON Data (User Transactions)
user_data = {
    "user_id": "12345",
    "transactions": [
        {"amount": 5000, "category": "Travel", "merchant": "Airline", "date": "2025-03-15"},
        {"amount": 1200, "category": "Dining", "merchant": "Restaurant", "date": "2025-03-10"},
        {"amount": 3500, "category": "Shopping", "merchant": "Luxury Store", "date": "2025-03-08"}
    ],
    "credit_score": 780,
    "current_card": "XYZ Bank Gold",
    "overdue": False,
    "credit_utilization": 40
}

credits_cards={
    "credits_cards": [
  {
    "card_name": "Elite Dining Rewards",
    "category": "Dining",
    "annual_fee": 999,
    "cashback": "5% on dining, 2% on groceries",
    "reward_points": "10x on restaurants",
    "benefits": [
      "Complimentary fine-dining vouchers",
      "Priority reservations at premium restaurants",
      "No surcharge on food delivery platforms"
    ],
    "min_income_required": 500000,
    "bank": "XYZ Bank"
  },
  {
    "card_name": "Global Travel Explorer",
    "category": "Travel",
    "annual_fee": 2999,
    "cashback": "5% on flight and hotel bookings",
    "reward_points": "15x on international spends",
    "benefits": [
      "Complimentary airport lounge access",
      "Zero forex markup",
      "Travel insurance and trip cancellation cover"
    ],
    "min_income_required": 700000,
    "bank": "ABC Bank"
  },
  {
    "card_name": "Luxury Prestige Platinum",
    "category": "Luxury",
    "annual_fee": 4999,
    "cashback": "3% on premium brands",
    "reward_points": "20x on luxury shopping",
    "benefits": [
      "Personalized concierge service",
      "Access to exclusive fashion events",
      "Free business-class flight upgrade once a year"
    ],
    "min_income_required": 1000000,
    "bank": "Premium FinCorp"
  },
  {
    "card_name": "Smart Shopper Cashback",
    "category": "Shopping",
    "annual_fee": 799,
    "cashback": "10% on Amazon, Flipkart, Myntra",
    "reward_points": "5x on all retail shopping",
    "benefits": [
      "Buy now, pay later with 0% interest",
      "Extra warranty on electronics",
      "Exclusive partner discounts"
    ],
    "min_income_required": 400000,
    "bank": "Retail Bank"
  },
  {
    "card_name": "Fuel Saver Gold",
    "category": "Fuel",
    "annual_fee": 499,
    "cashback": "5% on fuel transactions",
    "reward_points": "3x on fuel and toll payments",
    "benefits": [
      "1% fuel surcharge waiver",
      "Free road assistance and car insurance discounts",
      "Monthly fuel vouchers"
    ],
    "min_income_required": 300000,
    "bank": "Petro Finance"
  },
  {
    "card_name": "Online Spender Pro",
    "category": "Online",
    "annual_fee": 999,
    "cashback": "7% on OTT, food delivery & gaming",
    "reward_points": "8x on e-commerce & digital payments",
    "benefits": [
      "Free Netflix & Spotify subscription",
      "No-cost EMI on gadgets",
      "Cyber fraud protection"
    ],
    "min_income_required": 450000,
    "bank": "DigitalPay Bank"
  }
]
}


# Test model
messages = f"""
User Transactions: {json.dumps(user_data, indent=2)}
Credits Cards: {json.dumps(credits_cards, indent=2)}

### Question: You are a bank, which credit card would you recommend for this user based on their transaction history? Explain the customer how it benefits (explain only listed benefits above for that particular card) them in more marketing ways with numbers and all
"""

device = "cuda:0"
inputs = tokenizer(messages, return_tensors="pt").to(device)

outputs = base_model.generate(**inputs, max_new_tokens=400)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



User Transactions: {
  "user_id": "12345",
  "transactions": [
    {
      "amount": 5000,
      "category": "Travel",
      "merchant": "Airline",
      "date": "2025-03-15"
    },
    {
      "amount": 1200,
      "category": "Dining",
      "merchant": "Restaurant",
      "date": "2025-03-10"
    },
    {
      "amount": 3500,
      "category": "Shopping",
      "merchant": "Luxury Store",
      "date": "2025-03-08"
    }
  ],
  "credit_score": 780,
  "current_card": "XYZ Bank Gold",
  "overdue": false,
  "credit_utilization": 40
}
Credits Cards: {
  "credits_cards": [
    {
      "card_name": "Elite Dining Rewards",
      "category": "Dining",
      "annual_fee": 999,
      "cashback": "5% on dining, 2% on groceries",
      "reward_points": "10x on restaurants",
      "benefits": [
        "Complimentary fine-dining vouchers",
        "Priority reservations at premium restaurants",
        "No surcharge on food delivery platforms"
      ],
      "min_income_required": 500000,
    